# Saving Model Card in Onelake
Note: Run this notebook in Azure Machine Learning Severless Spark Compute

## Saving model card as csv in OneLake

In [ ]:
%pip install azure-ai-ml azure-identity pyspark delta-spark

## Save Model Card as delta table in Onelake

In [ ]:
#Execute through Serveress Spark Compute

# Install required packages:
# %pip install azure-ai-ml azure-identity pyspark delta-spark

from azure.ai.ml import MLClient
from azure.identity import ClientSecretCredential
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, MapType
from delta import configure_spark_with_delta_pip

# ---------------------------
# 2. Service Principal Credentials
# ---------------------------
tenant_id = "<tenant-id>"
client_id = "<client-id>"
client_secret = "<client-secret>"

credential = ClientSecretCredential(tenant_id, client_id, client_secret)

# ---------------------------
# 3. AML Workspace Details
# ---------------------------
SUBSCRIPTION = "<subscription-id>"
RESOURCE_GROUP = "rg-we-atpws-aml"
WS_NAME = "aml-ws-atp001"

ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

# ---------------------------
# 4. Get Registered Model
# ---------------------------
model = ml_client.models.get(name="credit_default_prediction_standard", version="1")

# ---------------------------
# 5. Prepare Model Card Metadata
# ---------------------------
model_card = {
    "model_name": model.name,
    "version": model.version,
    "description": model.description,
    "tags": model.tags,
    "properties": model.properties,
    "validation_status": model.tags.get("validation_status", "unknown"),
    "cost_info": model.tags.get("cost", "not provided")
}

# ---------------------------
# 6. Define Explicit Schema
# ---------------------------
schema = StructType([
    StructField("model_name", StringType(), True),
    StructField("version", StringType(), True),
    StructField("description", StringType(), True),
    StructField("tags", MapType(StringType(), StringType()), True),
    StructField("properties", MapType(StringType(), StringType()), True),
    StructField("validation_status", StringType(), True),
    StructField("cost_info", StringType(), True)
])

# ---------------------------
# 7. Create Spark DataFrame with Schema
# ---------------------------
model_card_df = spark.createDataFrame([model_card], schema=schema)

# ---------------------------
# 8. Save as Delta Table in OneLake
# ---------------------------
delta_path = "abfss://UnifiedData@onelake.dfs.fabric.microsoft.com/maag_bronze.Lakehouse/Tables/dbo/azureml_card_cardit_model_card"

model_card_df.write.format("delta").mode("overwrite").save(delta_path)

print(f"✅ Model card saved as Delta table at: {delta_path}")

StatementMeta(b1e6cefc-f7d3-40b2-90a7-9bef21816697, 6, 16, Finished, Available, Finished)

✅ Model card saved as Delta table at: abfss://UnifiedData@onelake.dfs.fabric.microsoft.com/maag_bronze.Lakehouse/Tables/dbo/model_card_delta
